In [1]:
import numpy as np
import time
import stock
import kline
import choose

In [4]:
def K(code):
    c,k,d = stock.loadKline(code)
    
    macdR = choose.macdRange(k)
    #扩大一个，并且放置向前溢出
    macdR[:,0]-=1
    if macdR[0,0]<0:
        macdR[0,0] = 0 
    pt = choose.intersectionIndexs([
        choose.rangeToIndexs(macdR),
        choose.rangeToIndexs(choose.bollSelectRange(k,20,0.2,15)),
        choose.volumeIndexs(k,1.5,20),
    ])
    pt = choose.rangeFirstIndexs(macdR,pt,len(k))    
    kline.Plote(code,'d',config={
    'macd':False,
    'vlines':[
        {
            'x':pt,
            'linewidth':2,
            'color':'red',
            'linestyle':'-.'
        }
    ]}).show()

In [5]:
K('SH:600460')

Box(children=(Button(description='上一页', style=ButtonStyle()), Button(description='下一页', style=ButtonStyle()), …

Output()

In [3]:
a = np.random.randn(100)

In [10]:
np.argwhere(a>0).reshape(-1)

array([ 0,  1,  3,  4,  5,  6,  9, 10, 11, 13, 14, 15, 17, 18, 19, 20, 23,
       24, 27, 30, 31, 32, 33, 35, 37, 38, 39, 41, 42, 46, 47, 49, 50, 53,
       54, 56, 57, 60, 63, 65, 66, 67, 69, 71, 72, 75, 76, 78, 79, 80, 82,
       83, 88, 89, 90, 93, 94, 96, 99], dtype=int64)

In [14]:
a = np.array([9,3,4,6,1])

In [8]:
p = np.zeros((100),dtype=np.bool)

In [6]:
a = np.array([9,3,4,6,1])

In [7]:
choose.flatIndexs(a,20)

array([False,  True, False,  True,  True, False,  True, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False])

In [13]:
p

array([False,  True, False,  True,  True, False,  True, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])